In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Yolov2(nn.Module):
  def __init__(self, num_classes=1):
    super(Yolov2, self).__init__()
    self.backbone = nn.Sequential(
        #conv1
        nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(32),
        nn.LeakyReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),

        #conv2
        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(64),
        nn.LeakyReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),

        #conv3
        nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(128),
        nn.LeakyReLU(inplace=True),

        #conv4
        nn.Conv2d(128, 64, kernel_size=1, stride=1, padding=0),
        nn.BatchNorm2d(64),
        nn.LeakyReLU(inplace=True),

        #conv5
        nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(128),
        nn.LeakyReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),

        #conv6
        nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(256),
        nn.LeakyReLU(inplace=True),

        #conv7
        nn.Conv2d(256, 128, kernel_size=1, stride=1, padding=0),
        nn.BatchNorm2d(128),
        nn.LeakyReLU(inplace=True),

        #conv8
        nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(256),
        nn.LeakyReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),

        #conv9
        nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(512),
        nn.LeakyReLU(inplace=True),

        #conv10
        nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=0),
        nn.BatchNorm2d(256),
        nn.LeakyReLU(inplace=True),

        #conv11
        nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(512),
        nn.LeakyReLU(inplace=True),

        #conv12
        nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=0),
        nn.BatchNorm2d(256),
        nn.LeakyReLU(inplace=True),

        #conv13
        nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(512),
        nn.LeakyReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),

        #conv14
        nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(1024),
        nn.LeakyReLU(inplace=True),

        #conv15
        nn.Conv2d(1024, 512, kernel_size=1, stride=1, padding=1),
        nn.BatchNorm2d(512),
        nn.LeakyReLU(inplace=True),

        #conv16
        nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(1024),
        nn.LeakyReLU(inplace=True),

        #conv17
        nn.Conv2d(1024, 512, kernel_size=1, stride=1, padding=1),
        nn.BatchNorm2d(512),
        nn.LeakyReLU(inplace=True),

        #conv18
        nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(1024),
        nn.LeakyReLU(inplace=True),

        #conv19
        nn.Conv2d(1024, 1024, kernel_size=3, stride=1, padding=1),
        #nn.BatchNorm2d(1024),
        #nn.LeakyReLU(inplace=True),

        #conv20
        nn.Conv2d(1024, 1024, kernel_size=3, stride=1, padding=0),
        #nn.BatchNorm2d(1024),
        #nn.LeakyReLU(inplace=True),

        #conv21
        nn.Conv2d(1024, 1024, kernel_size=3, stride=1, padding=0),
        #nn.BatchNorm2d(1024),
        #nn.LeakyReLU(inplace=True),

    )
    self.final_layer = nn.Conv2d(3072, 30, kernel_size=1, stride=1, padding=0)
    self.avg = nn.AvgPool2d((1,1))

  def forward(self, x):
    for i, layer in enumerate(self.backbone):

        x = layer(x)
        if i == 40:

           passthrough_layer = torch.cat( (x[:,:, 0:13,0:13] , x[:,:,0:13,13:26],
                              x[:,:,13:26,0:13], x[:,:,13:26,13:26]) , dim=1)

    x = torch.cat((passthrough_layer, x), dim=1)
    x = self.final_layer(x)
    x = self.avg(x)
    x = x.permute(0, 2, 3, 1)


    return x



In [ ]:
torch.log(torch.tensor(-1))

tensor(nan)

In [ ]:
import torch
import torch.nn as nn
import torchvision


class yolov2_main(nn.Module):
  def __init__(self, yolov2=None):
    super(yolov2_main, self).__init__()
    self.backbone = yolov2()

  def forward(self, images, targets=None, anchors=None):

    predictions = self.backbone(images)
    loss = self.calc_loss(predictions, targets, images, anchors)

    return loss

  def calc_loss(self, predictions, targets, images, anchors):

      ##Note:::: this model has no classifier for the reason that it addresses face detection, so
      ### objectness score is the major element

      pre_processing = output_preprocessing(predictions, targets, images, anchors)

      target_offsets = pre_processing[0]
      pred_offsets = pre_processing[1][:,0:4]


      loss_xy = F.mse_loss(nn.functional.sigmoid(pred_offsets[:,:2]), target_offsets[:,:2], reduction='sum')
      loss_wh = F.mse_loss(torch.exp(pred_offsets[:,2:4]), target_offsets[:,2:4], reduction='sum')
      offset_loss = (loss_xy + loss_wh )/target_offsets[:,0:4].numel() * 2


      neg_conf_scores = pre_processing[2]
      neg_conf_target = torch.zeros_like(neg_conf_scores).to(device=neg_conf_scores.device)
      negative_conf_loss = F.mse_loss(nn.functional.sigmoid(neg_conf_scores), neg_conf_target, reduction='sum')/neg_conf_target.numel() * 0.6



      pos_conf_scores = pre_processing[1][:, 4]
      pos_conf_target = torch.ones_like(pos_conf_scores,dtype=pos_conf_scores.dtype).to(device=pos_conf_scores.device)
      positive_conf_loss = F.mse_loss(nn.functional.sigmoid(pos_conf_scores), pos_conf_target, reduction='sum')/pos_conf_target.numel() * 0.5


      return (offset_loss + negative_conf_loss + positive_conf_loss), offset_loss, negative_conf_loss, positive_conf_loss






In [ ]:
from tqdm import tqdm
from functools import partial
tqdm = partial(tqdm, position=0, leave=True)
import torch
import numpy as np
import time
from PIL import Image
import matplotlib.pyplot as plt


def train_epocs(model, optimizer, data_loader, devie, all_anchors ,epochs=30,training_state=False ):

    for epoch in range(epochs):
        if epoch==30:
           parameters = model.parameters()
           optimizer = torch.optim.SGD(parameters, lr = 0.000002,
                                       momentum=0.99)
        total = 0
        sum_loss = 0
        sum_loss_classifier = 0
        sum_loss_offsets = 0
        sum_loss_noobj = 0
        sum_loss_obj = 0

        iteration_num = 0

        for data in tqdm(data_loader):
            optimizer.zero_grad()
            images = data[0].to(device)
            #targets = [ boxes[2].to(device) for boxes in data[1]]
            #targets = torch.stack(targets)
            targets = data[1]
            batch_length = len(images)
            ### gradient tracking
            if not training_state:
               with torch.no_grad():
                  model.eval()
                  output = model(images, targets = targets, anchors = all_anchors)
            else:
                 model.train()
                 output = model(images, targets, all_anchors)
            Final_loss = output[0]
            if training_state:
               optimizer.zero_grad()
               Final_loss.backward()
               optimizer.step()
            iteration_num += 1
            total += batch_length

            sum_loss += Final_loss
            #sum_loss_classifier += output[1]
            sum_loss_offsets += output[1]
            sum_loss_noobj += output[2]
            sum_loss_obj += output[3]

            if iteration_num % 50 == 0:

                train_loss = sum_loss/total
                train_loss_classifier = sum_loss_classifier/total
                train_loss_offsets = sum_loss_offsets/total
                train_loss_noobj = sum_loss_noobj/total
                train_loss_obj = sum_loss_obj/total
                print("loss %.6f classifer %.6f regressor %.6f noobj_loss %.6f obj_loss %.6f "%
                      (train_loss, train_loss_classifier, train_loss_offsets, train_loss_noobj, train_loss_obj))
                total = 0.0
                sum_loss= 0.0
                sum_loss_classifier = 0.0
                sum_loss_offsets = 0.0
                sum_loss_noobj = 0.0
                sum_loss_obj = 0.0


    return model


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = yolov2_main(yolov2=Yolov2).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4 , momentum = 0.99)
print(device)


cpu


In [ ]:
torch.s ave(model.state_dict(), '/content/gdrive/MyDrive/yolov2new.pth')





In [ ]:
model.load_state_dict(torch.load('/content/gdrive/MyDrive/yolov2new.pth', map_location=device))






<All keys matched successfully>

In [ ]:
train = train_epocs(model, optimizer, data_loader, device, all_anchors,epochs=1 ,training_state=True)




In [ ]:
import cv2
import matplotlib.pyplot as plt

def output_preprocessing(predictions, gt_boxes, images, anchors):
  target_offsets = []
  pred_offsets =[]
  neg_objectness_scores = []
  neg_objectness_scores_within = []


  for pred, gt in zip(predictions, gt_boxes):
    centroids = torch.tensor([ sample[1] for sample in gt ]).to(device=pred.device)
    boxes_in_image = torch.stack([ sample[2] for sample in gt]).to(device=pred.device)


    grid_cell_mask = torch.zeros((13,13)).to(device=pred.device)
    grid_cell_mask[centroids[:,1], centroids[:,0]] = 1
    mask_locations = torch.where(grid_cell_mask==0)
    masked_values = pred[ mask_locations[0],mask_locations[1],:]
    scores_locations = [4, 10, 16, 22, 28]

    #check_predicted_boxes = check(pred.detach().clone().to(device=torch.device('cpu')), gt,
                                  images[0].detach().clone().to(device=torch.device('cpu')), anchors)
    neg_objectness_scores.append(masked_values[:,scores_locations].view(-1))



    unique_centroids, counts = torch.unique(centroids, return_counts=True, dim=0)


    for grid_cell in unique_centroids:
        indices_of_boxes_in_cell_mask = torch.all(centroids==grid_cell, dim=1)
        indices_of_boxes_in_cell = torch.where(indices_of_boxes_in_cell_mask)[0]


        boxes_in_cell   = boxes_in_image[indices_of_boxes_in_cell]



        pred_boxes_in_cell = pred[ grid_cell[0], grid_cell[1],:].reshape(-1,6)

        anchors_in_cell = torch.tensor(anchors[ (grid_cell[0]*13) + grid_cell[1] ]).to(device=pred.device)

        assigned_anchors, assigned_bboxes, anchor_indices = anchors_assignmets( anchors_in_cell, boxes_in_cell )
        '''
        img = images[0].to(torch.device('cpu')).numpy().transpose(1,2,0).copy()
        for d in assigned_anchors:
          x1 = int(d[0])
          y1 = int(d[1])
          x2 = int(d[2])
          y2 = int(d[3])
          cv2.rectangle(img, (x1,y1), (x2,y2),(0,0,255), 2)

        for d in assigned_bboxes:
          x1 = int(d[0] - d[2]/2 )
          y1 = int(d[1] - d[3]/2)
          x2 = int(d[2]/2 + d[0])
          y2 = int(d[3]/2 + d[1] )
          cv2.rectangle(img,(x1, y1), (x2, y2), (255,0,0), 1)
        plt.imshow(img)
        print(er)
        '''
        assigned_pred = pred_boxes_in_cell[anchor_indices,:]

        for i in range(anchors_in_cell.size(0)):
            if i not in anchor_indices:
               neg_objectness_scores_within.append(pred_boxes_in_cell[i][-2])

        target_bbox = target_calculation(assigned_anchors, assigned_bboxes)

        target_offsets.append(torch.stack(target_bbox))
        pred_offsets.append(assigned_pred)


  negative_scores = torch.cat( (torch.cat(neg_objectness_scores), torch.stack(neg_objectness_scores_within)) )

  return torch.cat(target_offsets), torch.cat(pred_offsets).view(-1,6), negative_scores





In [ ]:
def target_calculation(assigned_anchors, assigned_bbox):
    target_boxes = []
    for anchor, box in zip(assigned_anchors, assigned_bbox):
        x = (box[0] - ( int(box[0]/32) * 32))/32
        y = (box[1] - ( int(box[1]/32) * 32))/32

        anchor_w = anchor[2] - anchor[0]
        anchor_h = anchor[3] - anchor[1]

        w = box[2]/anchor_w
        h = box[3]/anchor_h

        target = torch.tensor([x, y, w, h], dtype=torch.float32).to(box.device)
        target_boxes.append(target)

    return target_boxes



In [ ]:
def calc_iou(anchors, gt_boxes):
  anchors_area = (anchors[:,2] - anchors[:,0]) * (anchors[:,3]-anchors[:,1] )
  boxes_area = (gt_boxes[:,2]) * (gt_boxes[:,3])
  upper_left_intersection = torch.max(anchors[:,:2].unsqueeze(1), (gt_boxes[:,:2] - gt_boxes[:,2:]/2))
  lower_right_intersection = torch.min(anchors[:, 2:].unsqueeze(1), (gt_boxes[:,:2] + gt_boxes[:,2:]/2))
  intersection_dimension = (lower_right_intersection - upper_left_intersection).clamp(min=0)
  intersection_area = intersection_dimension[:,:,0] * intersection_dimension[:,:,1]
  union_area = anchors_area.unsqueeze(1) + boxes_area - intersection_area
  return intersection_area/union_area

def anchors_assignmets(anchors, gt):
    assigned_anchors = []
    assigned_bboxes = []
    anchors_indices = []

    if len(anchors) < len(gt):
       gt = gt[: len(anchors)]

    iou = calc_iou(anchors, gt)
    iou = torch.transpose(iou, 0, 1)

    for i, col in enumerate(iou):
        trig = 0
        values, indices = torch.sort(col, descending=True)
        for num in range(len(anchors_indices)):
          if indices[num] in anchors_indices:
            trig = num+1
          else:
            break
        anchors_indices.append(indices[trig])
        assigned_bboxes.append(gt[i])
        assigned_anchors.append(anchors[indices[trig]])

    return torch.stack(assigned_anchors), torch.stack(assigned_bboxes), anchors_indices




In [1]:
import torch

x = torch.arange(0,416,32)
print(x)

tensor([  0,  32,  64,  96, 128, 160, 192, 224, 256, 288, 320, 352, 384])


In [ ]:
from matplotlib.patches import BoxStyle
def check(pred, gt, image, anchors):
  pred = pred.view(13,13,5,6)
  prediction = convert_offsets_pred(pred, anchors)
  boxes_with_boxes = torch.where( nn.functional.sigmoid(pred[:,:,:,4]) > 0.8 )
  true_predictions = prediction[boxes_with_boxes]


  img = image.numpy().transpose(1,2,0).copy()
  for d in true_predictions:
    x1 = int( d[0] -d[2]/2 )
    y1 = int( d[1]- d[3]/2 )
    x2 = int( d[2]/2 + d[0])
    y2 = int( d[3]/2 + d[1])
    cv2.rectangle(img ,(x1, y1),(x2, y2), (255,0,0), 2)
  plt.imshow(img)
  print(nm)






In [ ]:

def convert_offsets_pred(pred, anchors):
  c = torch.arange(0,416,32)
  cx, cy = torch.meshgrid(c,c)

  bx = nn.functional.sigmoid(pred[:,:,:, 0])*32 + cx.unsqueeze(-1)
  by = nn.functional.sigmoid(pred[:,:,:, 1])*32 + cy.unsqueeze(-1)

  anchors = torch.tensor(anchors).view(13,13,5,-1)
  anchors[:,:,:,2] = anchors[:,:,:,2] - anchors[:,:,:,0]
  anchors[:,:,:,3] = anchors[:,:,:,3] - anchors[:,:,:,1]

  bw = anchors[:,:,:,2] * torch.exp(pred[:,:,:,2])
  bh = anchors[:,:,:,3] * torch.exp(pred[:,:,:,3])


  offsets = torch.cat((bx.unsqueeze(-1), by.unsqueeze(-1), bw.unsqueeze(-1), bh.unsqueeze(-1)), dim=-1)

  return offsets





Data uploading

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip /content/gdrive/MyDrive/wider_face_split.zip


Archive:  /content/gdrive/MyDrive/wider_face_split.zip
   creating: wider_face_split/
  inflating: wider_face_split/readme.txt  
  inflating: wider_face_split/wider_face_test.mat  
  inflating: wider_face_split/wider_face_test_filelist.txt  
  inflating: wider_face_split/wider_face_train.mat  
  inflating: wider_face_split/wider_face_train_bbx_gt.txt  
  inflating: wider_face_split/wider_face_val.mat  
  inflating: wider_face_split/wider_face_val_bbx_gt.txt  


In [ ]:
!unzip /content/gdrive/MyDrive/WIDER_train.zip

In [ ]:

from PIL import Image
import torchvision
import numpy as np
import cv2
import torch

class LoadDataSet(torch.utils.data.Dataset):
  def __init__(self, annotation_path, images_path, transforms):
    self.images_path_list = images_path
    self.annotation_path_list = annotation_path
    self.transform = transforms
    self.size = len(self.images_path_list)
    print(self.size)

  def __getitem__(self, idx):
    Transform = torchvision.transforms.Normalize(mean=[0.485,0.456,0.406],
                                                 std=[0.229,0.224,0.225])

    img_path = '/content/WIDER_train/images/'+self.images_path_list[idx]
    gt_boxes = self.annotation_path_list[idx]
    img = Image.open(img_path)
    img = np.asarray(img)
    img_size = img.shape
    image = cv2.resize(img/255., (416,416), interpolation=cv2.INTER_AREA)
    image = torch.as_tensor(image, dtype=torch.float32).permute(2,0,1)
    image = Transform(image)
    scalar = torch.tensor([1, 416/img_size[1], 416/img_size[0], 416/img_size[1], 416/img_size[0]], dtype=torch.float32)
    boxes = []
    scale_ratio = 32

    for box in gt_boxes:
        s_box = torch.tensor(box, dtype=torch.float32)*scalar

        x_ctr = (s_box[3] + s_box[1])/2
        w     = (s_box[3] - s_box[1])
        #print("x_ctr", x_ctr)
        x0_cell = int(x_ctr/scale_ratio)

        y_ctr = (s_box[4] + s_box[2])/2
        h     = (s_box[4] - s_box[2])
        #print("y_ctr", y_ctr)
        y0_cell = int(y_ctr/scale_ratio)
        '''
        if (x_ctr, y_ctr) in intersection and (y_ctr==0 or x_ctr==0):
          print(x_ctr, y_ctr)
          print(img_path)
        intersection.append((x_ctr, y_ctr))
        '''
        s_box[1] = x_ctr
        s_box[2] = y_ctr
        s_box[3] = w
        s_box[4] = h

        boxes.append([1, (x0_cell, y0_cell), s_box[1:]])

    return image, boxes, img_path

  def __len__(self):
    return self.size

def collate_fn(batch):
  images = []
  boxes = []
  for data in batch:
    images.append(data[0])
    boxes.append(data[1])
  return torch.stack(images, dim=0), boxes, data[2]





In [ ]:
file = open('/content/wider_face_split/wider_face_train_bbx_gt.txt')
lista=[]
inside_list=[]
names=[]
# this parameter is used to generate the anchors
all_boxes = []
triger=0

for idx, line in enumerate(file.readlines()):
    if line[-4:]=="jpg\n":
       names.append(line[:-1])
       if (inside_list==[] or triger==1) and idx!=0:
          names.pop(len(names)-2)
          inside_list = []
          triger = 0
          continue
       if idx!=0 and inside_list!=[]:
          lista.append(inside_list)
          inside_list = []
    if line[-4:]!='jpg\n' and len(line)>15:
       num_list = line.strip()
       num_str = num_list.split()
       x1 = int(num_str[0])
       y1 = int(num_str[1])
       x2 = int(num_str[0]) + int(num_str[2])
       y2 = int(num_str[1]) + int(num_str[3])
       if num_str[2]!='0' and num_str[3]!='0' and num_str[7]!='1' and (x2-x1)>20 and (x2-x1)<1200 and (y2-y1)>20 and (y2-y1)<1200:
          array_int = [1, x1, y1, x2, y2]
          all_boxes.append(array_int[1:])
          inside_list.append(array_int)
       else:
          triger=1
    if idx == 185183:
       if inside_list!=[] and triger==0:
          lista.append(inside_list)
       elif inside_list==[] and triger==1:
          names.pop()


In [ ]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
dataset = LoadDataSet(annotation_path=lista, images_path=names, transforms=None)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn, drop_last=True)


9074


k mean clustring


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv2
import torch
import copy
boxes = torch.tensor([[10,10,90,40], [80, 70, 120, 190], [250, 5, 300, 250], [70, 90, 350, 500],
                  [290, 200, 430, 350], [390, 310, 520, 470],  [320, 230, 370, 310], [15, 150, 260, 550], [70, 50, 500, 530],
                  [120, 300, 250, 450], [60, 50, 270, 280]]).to(float)

centroids = torch.tensor([[100, 150, 160, 210], [50,50,205,347],
                      [10, 5, 416, 407], [0, 0, 400, 200],[0, 0,140, 500]]).to(float)



img = np.random.randint(240, 255, (416,416,3)).astype('float32')

In [ ]:
import torch
bounding_boxes = torch.tensor(all_boxes).to(float)

In [ ]:
new_boxes = copy.deepcopy(boxes_for_anchors)
for i, box in enumerate(new_boxes):
  new_boxes[i] = [0,0, box[2]-box[0], box[3] - box[1]]

new_boxes = torch.tensor(new_boxes).to(torch.float32)

new_centroids = copy.deepcopy(centroids)
for i, box in enumerate(new_centroids):
  new_centroids[i] = torch.tensor([0, 0, box[2]-box[0], box[3]-box[1]]).to(float)

In [ ]:
def calc_iou_anchors(boxes, centroids):
  boxes_area = (boxes[:,2] - boxes[:,0]) * (boxes[:,3] - boxes[:,1])
  centroids_area = (centroids[:,2] - centroids[:,0]) * (centroids[:,3] - centroids[:,1])

  upper_left_intersection = torch.max(boxes[:,:2].unsqueeze(1), centroids[:,:2])
  lower_right_intersection = torch.min(boxes[:, 2:].unsqueeze(1), centroids[:,2:])

  intersection_dimension = (lower_right_intersection - upper_left_intersection).clamp(min=0)
  intersection_area = intersection_dimension[:,:,0] * intersection_dimension[:,:,1]
  union_area = boxes_area.unsqueeze(1) + centroids_area - intersection_area

  return intersection_area/union_area







In [ ]:
def calc_closest(new_boxes, new_centroids):
  iou = calc_iou_anchors(new_boxes, new_centroids)

  distance = 1 - iou
  _, indices = torch.min(distance, dim=1)

  closets = []
  for i in range(len(new_centroids)):
      closets.append(torch.where(indices == i)[0])

  return closets



In [ ]:
while True:
  check_centroid = copy.deepcopy(new_centroids)
  for i  in range(len(new_centroids)):
      closets = calc_closest(new_boxes, new_centroids)
      new_centroids[i] = torch.mean(new_boxes[closets[i]], dim=0)
  print(0)
  if new_centroids.equal(check_centroid):
     print(check_centroid)
     print(new_centroids)
     break


creating anchors after the k-mean clustring for the cetroids


In [ ]:
import torch
new_centroids = torch.tensor([[  0.0000,   0.0000,  16.0557,  27.2696],
        [  0.0000,   0.0000,  30.4325,  49.2445],
        [  0.0000,   0.0000, 180.2798, 241.7805],
        [  0.0000,   0.0000,  53.1910,  86.6925],
        [  0.0000,   0.0000,  97.2692, 146.4186]], dtype=torch.float32)

In [ ]:
'''
for box in new_boxes:
  cv2.rectangle(img,  ( int(box[0]), int(box[1]) ), ( int(box[2]), int(box[3]) ),
                (0,0,255), 1)
'''
all_anchors = []

for i in range(13):
  for j in range(13):
    anchors_per_cell = []
    #if i == 6 and j==6:
    for box in new_centroids:
        '''
        cv2.rectangle(img, ( (i)*32 + 16 - int( (box[2]-box[0])/2  ) , (j)*32 + 16-int( (box[3]- box[1])/2 ) ),
               ((i)*32 +16 + int( (box[2] - box[0])/2 ), (j)*32 +16+ int( (box[3] - box[1])/2 )), (0,255,0),2)
        print([(i)*32 + 16 -int((box[2]-box[0])/2), (j) * 32 + 16 - int((box[3] - box[1])/2),
             (i)*32 +16 + int((box[2]-box[0])/2), (j)*32 +16 +int((box[3]-box[1])/2)])
        '''
        anchors_per_cell.append([ (i)*32 + 16 -int((box[2]-box[0])/2), (j)*32 + 16 - int((box[3]-box[1])/2),
                               (i)*32 + 16 + int((box[2]-box[0])/2), (j)*32 + 16+ int((box[3]-box[1])/2)])


    all_anchors.append(anchors_per_cell)
    #if j==12 and i==12:
      #break
  #if j==12 and i==12:
    #break

#plt.imshow(img)





In [ ]:
len(all_anchors)

169